# 초기세팅

## 모듈 임포트

In [1]:
from typing import List, Dict, Optional, TypedDict, Any
from fastapi import FastAPI , Body ,HTTPException, APIRouter, Query 
import torch
import ast
import json
import uuid
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer
from google.genai import types
from fastapi.middleware.cors import CORSMiddleware
import os
from langgraph.graph import StateGraph, END

## 모델 로딩

### 재미나이 로딩

In [2]:
# API 키 로딩
keys = []
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                keys.append(value)

# API 키 로드 및 환경변수 설정
load_api_keys('api_key.txt')

In [3]:
def keyChange():
    global client
    from google import genai
    tt = keys.pop(0)
    keys.append(tt)
    client = genai.Client(api_key=tt)

keyChange() #무료 키 다썻으면 다른 키로 로딩하기

### 백터DB & 임베딩 모델 로딩

In [4]:
model_vector = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 2. 원격 pgvector 연결 설정
def get_db_connection():
    db_info= dict()
    with open("db_info.txt","r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=",1)
                db_info[key] = value

    conn = psycopg2.connect(
        host=db_info['host'],
        database=db_info['database'],
        user=db_info['user'],
        password=db_info['password'],
        port=db_info['port']
    )
    register_vector(conn)
    return conn


## State

In [5]:
class State(TypedDict, total = False):
    txt: str  #front
    ep_num : int  #front 
    subtitle : str #front
    check : dict
    work_id : int    #front
    user_id : str    #front
    인물 : dict
    세계 : dict
    장소 : dict
    사건 : dict
    물건 : dict
    집단 : dict
    


# 기능

## 백터 DB

### 임베딩 백터 : 택스트 데이터를 임베딩 벡터로 바꿔서 반환(768차원)

In [6]:
def get_vector(text):
    return model_vector.encode([text])[0]

### 백터 DB 검색(유사도)  
(카테고리 or *(전부), 유저 질문, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [7]:

def userQuestion(category,user_query,user_id,work_id,sim,limit):  # 태그(*,인물,장소,사건,세계,등)
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    search_sql = """
    SELECT * FROM (
        SELECT 
            id,
            category,
            setting, 
            ep_num,
            1 - (embedding <=> %s::vector) AS similarity 
        FROM lorebooks 
        WHERE embedding IS NOT NULL 
            AND %s = ANY(user_id)
            AND work_id = %s
            AND (%s = '*' OR category = %s)
    ) sub
    WHERE similarity >= %s  -- 여기서 원하는 유사도 문턱값을 설정 (예: 0.3 이상)
    ORDER BY similarity DESC  
    LIMIT %s;
    """
    cur.execute(search_sql, (query_vector,user_id,work_id,category,category,sim,limit))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

In [11]:
# userQuestion("인물","케인의 직장 상사","ssssssss",15,0.3,5)

### DB 검색(일반)  
(카테고리, 검색할 대상, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [12]:
def select(category,keyword,user_id,work_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor()
    search_sql= """
    SELECT 
        id,
        setting,
        ep_num
    FROM lorebooks 
    WHERE %s = ANY(user_id)
        AND work_id = %s
        AND category = %s
        AND keyword = %s
    """
    cur.execute(search_sql, (user_id,work_id,category,keyword))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result

In [13]:
# select("인물","케인","ssssssss",15)

### DB 업로드(설정집 신규)

In [14]:
def lorebooks_insert(universe_id,work_id,user_id,setting):
    conn = get_db_connection()
    cur = conn.cursor()
    for tag, values in setting.items():
        for val in values:
            if not val:
                continue
            ep_num = val.pop('ep_num',[])
            keyword,data = val.popitem()
            embd = get_vector(f'{keyword} : {data}')
            try:
                insert_sql = """
                    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding,created_at,updated_at)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,CURRENT_TIMESTAMP,CURRENT_TIMESTAMP)
                    """
                cur.execute(insert_sql, (
                        universe_id, 
                        work_id,
                        [user_id],
                        keyword,
                        tag,
                        ep_num,
                        json.dumps({keyword:data}), 
                        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
                    ))
                conn.commit()
                print("성공적으로 저장되었습니다!")
            except Exception as e:
                print(e)
                return "500 Internal Server Error"  # DB연결실패
    cur.close()
    conn.close()

# 수동 신규 설정집 업로드
def db_insert(universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    insert_sql = """
    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_sql, (
        universe_id,
        work_id,
        user_id,
        keyword,
        category,
        ep_num,
        json.dumps(setting),
        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
    ))
    conn.commit()
    cur.close()
    conn.close()
    print("성공적으로 저장되었습니다!")

### DB 수정(설정집 결합)

In [15]:
def lorebooks_update(setting):
    for lore_id,result,ep_num in setting:
        conn = get_db_connection()
        cur = conn.cursor()
        new_vector = f"{result}"
        embedding_vector = get_vector(new_vector)
        update_sql = """
        UPDATE lorebooks 
        SET 
            ep_num = %s,
            setting = %s,
            embedding = %s,
            updated_at = CURRENT_TIMESTAMP
        WHERE id = %s;
        """
        try:
            cur.execute(update_sql, (ep_num,json.dumps(result), embedding_vector, lore_id))
            conn.commit()
            print(f"데이터가 성공적으로 수정되었습니다.")
        except Exception as e:
            conn.rollback()
            print(e)
            return (f"수정 실패: {e}")
        finally:
            cur.close()
            conn.close()


# DB 수동 수정
def db_update(lore_id,universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    embedding_vector = get_vector(new_vector)
    update_sql = """
    UPDATE lorebooks 
    SET 
        universe_id = %s,
        work_id = %s,
        user_id = %s,
        keyword = %s,
        category = %s,
        ep_num = %s,
        setting = %s,
        embedding = %s,
        updated_at = CURRENT_TIMESTAMP
    WHERE id = %s;
    """
    try:
        cur.execute(update_sql, (universe_id, work_id, user_id, keyword, category, ep_num, json.dumps(setting), embd, lore_id))
        conn.commit()
        print(f"데이터가 성공적으로 수정되었습니다.")
    except Exception as e:
        conn.rollback()
        return (f"수정 실패: {e}")
    finally:
        cur.close()

### 연관인물 검색

In [16]:
def relationship_select(target,work_id,user_id): #태그,키워드,타이틀,작가 4개 다있어야함
    if target != '*':
        conn = get_db_connection()
        cur = conn.cursor() #쿼리문 확인하고 수정해야함.
        search_sql= """
        SELECT 
        
        -- 매칭되는 인물관계 객체만 JSON 형태로 출력
        jsonb_build_object(
            l.keyword, relation
        ) AS matching_relation
    FROM 
        
        lorebooks l,
        jsonb_each(l.setting) AS person, -- 1단계: 인물명(Key)별로 분리
        jsonb_array_elements(person.value->'인물관계') AS relation -- 2단계: 인물관계 배열 전개
    WHERE 
        l.work_id = %s  -- 입력받을 work_id
        AND %s = ANY(l.user_id)  -- 입력받을 user_id
        AND (relation->>'대상이름' = %s OR relation->>'대상' = %s) -- 검색하려는 대상인물 이름
        AND l.category = '인물'"""
        cur.execute(search_sql, (work_id,user_id,target,target))
        result = cur.fetchall()
        # print(result)
        cur.close()
        conn.close()
        meinfo = select("인물",target,user_id,work_id)
        meinfo = [{target:meinfo[0][1][target]['인물관계']}]
        return result+meinfo
    else:
        conn = get_db_connection()
        cur = conn.cursor() #쿼리문 확인하고 수정해야함.
        search_sql= """
        SELECT 
        
        -- 매칭되는 인물관계 객체만 JSON 형태로 출력
        jsonb_build_object(
            l.keyword, relation
        ) AS matching_relation
    FROM 
        
        lorebooks l,
        jsonb_each(l.setting) AS person, -- 1단계: 인물명(Key)별로 분리
        jsonb_array_elements(person.value->'인물관계') AS relation -- 2단계: 인물관계 배열 전개
    WHERE 
        l.work_id = %s  -- 입력받을 work_id
        AND %s = ANY(l.user_id)  -- 입력받을 user_id
        AND l.category = '인물'"""
        cur.execute(search_sql, (work_id,user_id))
        result = cur.fetchall()
        # print(result)
        cur.close()
        conn.close()
        return result

### 사건검색

In [17]:
def timeline_select(target,work_id,user_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor() #쿼리문 확인하고 수정해야함.
    search_sql= """
SELECT 
    id,
    ep_num,
    keyword
    
FROM 
    lorebooks 
WHERE 
    work_id = %s  -- 입력받을 work_id
    AND %s = ANY(user_id)  -- 입력받을 user_id
    AND ep_num && %s
    AND category = '사건'
ORDER BY ep_num[1] ASC;
    """
    cur.execute(search_sql, (work_id,user_id,target))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result

## LLM 

### 메시지 입력

In [18]:
count_change= 0
models = ["gemini-2.5-flash-lite", 'gemini-3-flash',"gemini-2.5-flash","gemma-3-27b-it","gemini-3-flash-preview","gemini-2.5-pro",
         "gemini-2.5-flash-preview-09-2025","gemini-2.5-flash-lite-preview-09-2025",]
def modelchange():
    global mod
    mod = models.pop(0)
    models.append(mod)
modelchange()    
def genai(system_prompt,msg):
    global count_change
    try:
        if mod == "gemma-3-27b-it":
            response = client.models.generate_content(
                model=mod,
                contents=system_prompt+msg,
            )
            return response.text
        else:
            response = client.models.generate_content(
                model= mod,
                config=types.GenerateContentConfig(
                    system_instruction=system_prompt, # 여기에 시스템 프롬프트를 넣습니다.
                    temperature=0.1, # 데이터 추출이므로 낮게 설정
                ),
                contents=msg
            )
            return response.text
    except Exception as e:
        count_change +=1
        if count_change >=30:
            modelchange()
            count_change = 0
        keyChange()
        return genai(system_prompt,msg)


### 카테고리 추출

In [19]:
def Categories(state: State) -> State:
    system_prompt="""### Role
너는 소설 원문에서 설정 데이터를 누락 없이 추출하여 구조화하는 전문 데이터 분석가입니다.

### Task
당신은 '본문'의 글에서 설정집을 생성하기 쉽도록 키워드를 추출합니다. 
추출할 정보는 '인물, 세계, 장소, 사건, 물건, 집단'에 관한 내용이며, 설정집에 필요한 필수적인 정보들만 추출할 것입니다.
이를 딕셔너리{} 형태로 구조화하세요.

### Extraction Guidelines (카테고리별 정의)
- **인물** 
이름이 언급되거나 고유한 역할이 있는 등장인물(단순 엑스트라 제외)
- **세계**
마법 체계, 정치 시스템, 신화적 배경, 물리적 법칙 등 세계관을 이해하기에 필수적인 설정과 개념
- **장소**
사건이 벌어지는 지리적 배경, 건물 이름, 특정 지역
- **사건**
스토리에 영향을 끼칠 수 있는 구체적인 변화, 전투, 조약, 과거의 사건 등
- **물건**
고유 명칭이 있는 도구, 무기, 유물, 중요한 상징적 아이템
- **집단**
가문, 국가, 단체 등 2인 이상의 고유명칭을 가진 결사체

### Constraints
- 특정 주인공 중심의 요약을 하지 마세요. 등장하는 키워드를 분류하세요.
- 결과물에는 불릿 포인트, 볼드체, 추가 설명을 절대 포함하지 말고 순수 JSON/Dict 형식만 출력하세요.
- **본문에 명확히 나오지 않은 정보는 빈 배열 `[]`로 처리하세요.
- **수식어 제거**: 대상을 특정하기 위한 수식어(예: 'oooo의 정치체계','oo의 악마','oo왕국 기사단','oooo 제국')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하고 핵심 키워드만 추출하세요.
- **개체명 표준화: 동일 대상을 지칭하는 이명(예: '몰락한 왕'과 '비에고','비에고 국왕'과 '비에고' > 두 경우 다 '비에고'로 출력)은 가장 공식적인 명칭 하나로 통합하세요.

### Output Format 
{
'인물':['인물1','인물2',...],
'세계':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'물건':['물건1',....],
'집단':['집단1','집단2'],
}

"""
    msg = f"본문: {state.get('txt','')} \n 위 본문을 분석하여 '인물, 세계, 장소, 사건, 물건, 집단' 정보를 추출해줘."
    
    result = genai(system_prompt,msg)
    print("목록 추출 완료")
    print("=="*8)
    print(result)
    print("=="*8)
    try:
        st = result.index('{')
        ed = result.index('}')
        result = ast.literal_eval(result[st:ed+1])
    except:
        print("딕셔너리 변환 에러")
        # state = Categories(state)
    return {**state,
            'check' : result
           }

### 1.설정 추출 - 인물

In [42]:
def charter(state: State) -> State:
    lis = state.get('check')
    lis = lis['인물']
    txt = state.get('txt')
    chart = state.get('인물',{})
    for i in range(0, len(lis), 2):
        chunk = lis[i:i+2]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 인물 설정집(Character Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 등장인물명 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
각 인물별로 독립된 키를 가진 딕셔너리 형태로 출력해야 합니다
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요
# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{ "인물명1" : 
    { "별명": ["본문에 명시된 별명 (없으면 None)"],
      "배경": ["간략한 배경 스토리(없으면 None)"],
      "종족": ["종족 (없으면 None)"],
      "연령": ["나이 또는 연령대 (없으면 None)"],
      "직업/신분": ["신분 (없으면 None)"],
      "소속집단/가문": ["소속 (없으면 None)"],
      "외형": ["외모 묘사 (없으면 None)]",
      "성격": ["성격 키워드1 (없으면 None)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 None)", "숙련도/강도": "수준 (없으면 None)", "상세효과": "효과설명 (없으면 None)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 None)", "관계": "관계 정의( 없으면 None"), "상세내용": "관계 설명(없으면 None)"}
      ]
    },
    "인물명2" : 
    { "별명": ["본문에 명시된 별명 (없으면 None)"],
      "배경": ["간략한 배경 스토리(없으면 None)"],
      "종족": ["종족 (없으면 None)"],
      "연령": ["나이 또는 연령대 (없으면 None)"],
      "직업/신분": ["신분 (없으면 None)"],
      "소속집단/가문": ["소속 (없으면 None)"],
      "외형": ["외모 묘사 (없으면 None)]",
      "성격": ["성격 키워드1 (없으면 None)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 None)", "숙련도/강도": "수준 (없으면 None)", "상세효과": "효과설명 (없으면 None)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 None)", "관계": "관계 정의( 없으면 None"), "상세내용": "관계 설명(없으면 None)"}
      ]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 등장인물이 하나만 있을 경우에는 한 개체만 출력하세요.
- 제공한 등장인물 2명을 제외한 나머지는 추가적으로 출력하지 마세요.
- 인물관계에 소속집단이 들어가는 등 잘못된 정보가 들어가지 않도록 주의하세요."""
        
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                chart[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
        
    return {**state,
           '인물' :chart
           }

### 2.설정추출 - 세계관

In [41]:
def rule(state: State) -> State:
    lis = state.get('check')
    lis = lis['세계']
    txt = state.get('txt')
    rule = state.get('세계',{})
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 세계관 설정집을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 세계관 키워드 [{names_str}] 각각의 정보을/를 본문 없이 작품 속 세계의 규칙을 이해할 수 있도록 300자 이내로 설명하는 문장을 만들어 JSON 포맷으로 출력하세요.

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 요약은 가능하지만 본문의 내용을 바탕으로 추측 없이 문장을 생성하세요.
2. 연관된 이야기 없이 대상에 대한 내용만으로 짧게 요약하여 문장을 생성하세요.
3. 종류는 마법 체계, 정치 시스템, 신화적 배경, 물리적 법칙 등 어떤 부분을 설명하는지를 말합니다. 종류에는 값을 하나만 넣으세요.
"""+"""   
# JSON_Output_Format
{
"세계관 키워드1" : {
    "종류" : ["세계관 종류(없으면 None)"],
    "설명" : ["세계관 1 설명None"]},
"세계관 키워드2" : {
    "종류" : ["세계관 종류(없으면 None)"],
    "설명" : ["세계관 1 설명None"]},....
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 명확히 나오지 않은 정보는 None값 또는 빈 배열 `[]`로 처리하세요.
- 만약 세계관 키워드가 하나만 있을 경우에는 한 세계관만 출력하세요.
- 제공한 키워드를 제외한 나머지는 추가적으로 출력하지 마세요.
"""
        
        msg= f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."           
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                rule[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '세계' :rule
           }

### 3.설정추출 - 장소

In [43]:
def place(state: State) -> State:
    lis = state.get('check')
    lis = lis['장소']
    txt = state.get('txt')
    place = state.get('장소',{})
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]  
        names_str = ", ".join(chunk)
        system_prompt=  f"""# Role
귀하는 소설의 원문을 분석하여 장소 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 장소 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"장소 키워드1": 
    { "별칭": ["지역별칭 (없으면 None)"],
      "분위기": ["지역의 분위기(없으면 None)"],
      "역사": ["지역의 역사 또는 생성된 시기 (없으면 None)"],
      "위치": ["지역의 위치 (없으면 None)"],
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 None)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 None)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 None)"]
    },
 "장소 키워드2": 
    { "별칭": ["지역별칭 (없으면 None)"],
      "분위기": ["지역의 분위기(없으면 None)"],
      "역사": ["지역의 역사 또는 생성된 시기 (없으면 None)"],
      "위치": ["지역의 위치 (없으면 None)"],
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 None)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 None)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 None)"]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 None값으로 남겨두세요.
- 만약 장소 키워드가 하나만 있을 경우에는 한 장소만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요."""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
                    

        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                place[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '장소' :place
           }

### 4.설정추출 - 사건

In [44]:
def event(state: State) -> State:
    lis = state.get('check')
    lis = lis['사건']
    txt = state.get('txt')
    event = state.get('사건',{})
    for i in range(0, len(lis), 5):
        chunk = lis[i:i+5]  
        names_str = ", ".join(chunk)
        system_prompt= f"""# Role
귀하는 소설의 원문을 분석하여 사건 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 사건 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"사건 키워드1": {
    "관련 인물": ["관련 인물(없으면 None)"],
    "설명" : ["해당 사건에 관한 짧은 설명"]},
"사건 키워드2": {
    "관련 인물": ["관련 인물(없으면 None)"],
    "설명" : ["해당 사건에 관한 짧은 설명"]},
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 키워드가 하나만 있을 경우에는 한 사건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg=f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                event[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '사건' :event
           }

### 5.설정추출 - 아이템

In [45]:
def item(state: State) -> State:
    lis = state.get('check')
    lis = lis['물건']
    txt = state.get('txt')
    item = state.get('물건',{})
    
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 아이템 설정(item Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 아이템 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 물건의 성능과 역할에 대해 간략하게 적어야 합니다.
character에는 해당 사건의 메인 인물을 적어야 합니다.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"아이템 키워드1": {
    "설명":["해당 물건 관한 짧은 설명 (없으면 None)"],
    "관련인물" :["관련된인물 (없으면 None)"]},
"아이템 키워드2": {
    "설명":["해당 물건 관한 짧은 설명 (없으면 None)"],
    "관련인물" :["관련된인물 (없으면 None)"]}
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 관련인물은 사건에 등장하는 모든 인물이 아닌 아이템과 관련된 주역만 적어야 합니다. 주역이 없을 경우 빈 배열 '[]'로 처리하세요.
- 만약 키워드가 하나만 있을 경우에는 한 물건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                item[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '물건' :item
           }

### 6.설정추출 - 단체

In [46]:
def group(state: State) -> State:
    lis = state.get('check')
    lis = lis['집단']
    txt = state.get('txt')
    group = state.get('집단',{})
    
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]
        names_str = ", ".join(chunk)
        system_prompt=f"""# Role
귀하는 소설의 원문을 분석하여 집단 설정(group Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 집단 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 집단의 역사와 목적 대해 간략하게 적어야 합니다.
character에는 해당 집단에 속해있는 인물을 적은 후 집단에서 역할을 괄호'()'안에 적으세요.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"집단 키워드1": {
    "설명":["집단에 관한 짧은 설명(없으면 None)"],
    "구성원" :["구성원이름(역할)(없으면 None)"] },
"집단 키워드2": {
    "설명":["집단에 관한 짧은 설명(없으면 None)"],
    "구성원" :["구성원이름(역할)(없으면 None)"] },
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 None값으로 남겨두세요.
- 구성원은 대상을 특정하기 위한 수식어(예: '데마시아의 정치체계','총의 악마','OO왕국 기사단')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하여 넣으세요.
- 만약 키워드가 하나만 있을 경우에는 한 집단만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                group[k] = v

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '집단' :group
           }

### 관계도 다이어그램 출력

In [26]:
def Relationship(rel):
    system_prompt=f"""### Role
당신은 소설의 데이터를 분석하여 인물 관계를 최적화된 Mermaid 코드로 시각화하는 전문가입니다.

### Task
입력된 인물 데이터를 바탕으로 다음 '관계 통합 로직'에 따라 Mermaid 코드를 생성하세요.

### Core Logic (관계 통합 규칙)
1. 인물 식별 및 통합:
   - 동일 인물이 다른 이름으로 표기된 경우 하나로 통합하세요 (예: '소나', '소녀 소나' -> '소나').
   - 각 인물에 고유 ID(A, B, C...)를 부여하여 코드를 작성하세요. (형식: A[인물명])

2. 선 통합 및 양방향 기호 (<-->):
   - A와 B가 서로 주고받는 동일한 관계(예: A->B 동료, B->A 동료)는 반드시 하나로 합쳐 양방향 화살표(<-->)로 표시하세요.
   - 예: A[소나] <-->|동료| B[코로바크]

3. 추가 단방향 관계 (-->):
   - 양방향 관계 외에 특정 인물이 상대에게만 가지는 별도의 관계가 있다면 단방향 화살표(-->)를 추가하세요.
   - 예: A <-->|동료| B 연 결 후, B가 A를 짝사랑한다면 B -->|짝사랑| A 추가.

4. 선 개수 제한 (Maximum 3):
   - 인물 A와 B 사이에는 어떤 경우에도 선(Edge)이 '3개'를 초과해서는 안 됩니다.
   - 관계가 너무 많다면 중요도가 낮은 관계는 생략하거나, 하나의 라벨 안에 콤마(,)로 합쳐서 표현하세요.

### Constraints
- 출력 결과에는 제목, 설명, 불릿 포인트, ``` 등을 절대 포함하지 마세요.
- 오직 "graph TD"로 시작하는 Mermaid 코드만 출력하세요.
- 인물명과 관계의 공백은 언더바(_)로 대체하세요.
- 관계가 None인 경우 문맥상 가장 적절한 관계로 추측하여 기입하세요.

### 출력 예시
graph TD
    A[소나] <-->|동료,보호_대상| B[코로바크]
    B -->|희생하려_함| A
    B -->|심문_대상| C[케인]
"""
    msg = f"{rel}인물 관계 데이터가 이렇게 저장되어 있을 때 mermaid 코드로 출력해줘."
    
    result = genai(system_prompt,msg)
    print("관계도 출력 완료")
    print(result)
    return result



### 사건 타임라인 출력

In [27]:
def timeline(rel):
    system_prompt=f"""### Role
당신은 소설의 데이터를 분석하여 사건을 최적화된 Mermaid 코드로 시각화하는 전문가입니다.

### Task
입력된 인물 데이터를 바탕으로 다음 '출력 로직'에 따라 Mermaid 코드를 생성하세요.

### Core Logic (출력 로직)
1. 입력되는 데이터의 형식은 [(id, [ep_num], 키워드),.....]입니다.
2. 동일한 시간대에 키워드가 여러개 있다면 입력 데이터의 id 값을 참고하여 더 낮은값을 가진 키워드가 먼저 배치되게 하세요.
예시) 
입력 데이터 : [(2, [1], 키워드2),(1, [1], 키워드1),(3, [2], 키워드3),....]

출력 데이터 :
timeline
    title 사건 전계도
    1화 : 키워드1
        : 키워드2
    2화 : 키워드3
    ...

### Constraints
- 출력 결과에는 제목, 설명, 불릿 포인트, ``` 등을 절대 포함하지 마세요.
- 오직 "timeline"로 시작하는 Mermaid 코드만 출력하세요.

### 출력 예시
timeline
    title 사건 전계도
    ep_num화 : 키워드
    ep_num화 : 키워드
             : 키워드
    ep_num화 : 키워드
    ep_num화 : 키워드
"""
    msg = f"{rel}사건의 키워드를 찾은 데이터가 이렇게 저장되어 있을 때 mermaid 코드로 출력해줘."
    
    result = genai(system_prompt,msg)
    print("타임라인 출력 완료")
    print(result)
    return result



### 추출한 설정 검수 & 분류

In [39]:
def comparison(state: State):
    check = state.get('check')
    print(state)
    work_id = state.get('work_id')
    user_id = state.get('user_id')
    uplist ={'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    editlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    errorlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    oldlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    for tag,val in check.items():
        for keyword in val:
            emb = select(tag,keyword,user_id,work_id)
            if (len(emb)==0):
                uplist[tag].append({keyword:state.get(tag).get(keyword),"ep_num":[state.get("ep_num")]}) 
                continue
            elif tag == "사건":
                if (len(emb)!=0):
                    errorlist[tag].append({keyword:"[결과: 충돌]\n [판단사유: 사건 이름이 동일한 사건이 존재합니다. 키워드를 변경해주세요.]","신규설정":state.get(tag).get(keyword)})
                    oldlist[tag].append(emb)
                    continue
                else:
                    uplist[tag].append({keyword:state.get(tag).get(keyword),"ep_num":[state.get("ep_num")]}) 
                    continue
            oldlist[tag].append(emb)
            i = state.get(tag).get(keyword)
            system_prompt= f"""#### 역할: 소설 설정 검수 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 설정이 어긋나는 경우가 존재하는지 찾아보세요.

### 분석할 설정
기존설정 : {emb[0][1]}

신규설정 : {i}

### 요청 사항
2. **상세 분석:** - **인과관계/상세화:** 
   - **결합:** 한쪽이 원인이고 다른 쪽이 결과이거나, 한쪽이 다른 쪽에서 없는 설명인 경우 '결합'로 판단합니다.
   - **충돌:** 예외 없이 물리적으로 불가능한 경우에만 '충돌'로 판단하세요.
    -**일치:** 설정이 추가되거나 보강되는 부분 없이 동일한 경우에는 '일치'로 판단하세요.
    
### 출력 형식
[결과: 충돌/결합/일치]
[판단사유: 판단사유]
        """
            msg = f"이거 설정이 맞을까?"     
            result = genai(system_prompt,msg)
            if "[결과: 충돌]" in result:
                errorlist[tag].append({keyword:result,"신규설정":i})
            elif "[결과: 결합]" in result:
                editlist[tag].append(emb)
            print(result)
    editlist = edit_setting(editlist,state)
    return {'충돌': errorlist, '설정 결합' : editlist,
            "신규 업로드": uplist, "기존설정" : oldlist}

In [29]:
def edit_setting(editlist, state:State):
    lis = []
    for tag,value in editlist.items():
        for emb in value:            
            vctid,oldset,episode = emb[0]
            keyword = next(iter(oldset))
            system_prompt= f"""#### 역할: 소설 설정 작성 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 두 설정을 결합하세요.

### 분석할 설정
기존설정 : {oldset}

신규설정 : {keyword} : {state.get(tag).get(keyword)}

### 제한 사항
-기존 설정과 신규 설정의 시점 차이가 있을 경우, 시간상으로 나중에 일어난 일으로 업데이트하세요.
(예: 16세일때 외형묘사가 흑발, 18세일때 외형묘사가 금발이라면 외형을 금발으로 업데이트)
-서로 상충되는 부분이 아니라 보강되는 부분이라면 두 설정을 결합하세요.
(예: 기존 설정- 외형: 근육질, 신규 설정 -외형 : 흑발, > 결합 설정- 외형 : 근육질, 흑발
     설정1 -인물관계: 동료, 설정2-인물관계:오랜 친구 > 결합설정 - 인물관계 : 동료이자 오랜 친구)
- 서로 결합할 때, 키값을 임의로 변경하지 마세요.
### 출력 형식
신규설정 형식과 동일
"""
            msg = f"두 설정을 결합해줄래"     
            result = genai(system_prompt,msg)
            try:
                result = result.replace("json", "").replace("```", "").strip()
                result = json.loads(result)
            except:
                print("딕셔너리 변환 에러 : edit_setting")
                print(result)
                #state = charter(state)
            print(result)
            episode.append(state.get('ep_num',0))
            lis.append([vctid,result,episode])
    return lis

## 원문&설정 읽기 쓰기

In [30]:
def save_text(user_id, work_id, ep_num, txt):
    try:
        save_dir = f"./save_original/{user_id}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{ep_num}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(txt)
            f.close()
        return f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
    except Exception as e:
        print("error occured: ", e)
        return f"failed"

def open_text(user_id, work_id, ep_num) -> str:
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e: 
        print("error occured: ", e)
        return "failed"
        
def save_json(user_id, work_id,lore_id, data):
    try:
        save_dir = f"./save_setting/{userid}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{lore_id}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return f"./save_setting/{userid}/{work_id}/{lore_id}.json"
    except Exception as e:
        print("error occured: ", e)
        return "failed"

def open_json(user_id, work_id,lore_id) :
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{lore_id}.json"
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e: 
        print("error occured: ", e)
        return "failed"

## Rang Graph

### 카테고리

In [31]:
cat_builder = StateGraph(State)
cat_builder.add_node("Categories",Categories)

cat_builder.set_entry_point("Categories")
cat_builder.add_edge("Categories",END)

cat = cat_builder.compile()

### 상세설정

In [32]:
builder = StateGraph(State)
builder.add_node("charter",charter)
builder.add_node("rule",rule)
builder.add_node("place",place)
builder.add_node("event",event)
builder.add_node("item",item)
builder.add_node("group",group)

builder.set_entry_point("charter")
builder.add_edge("charter","rule")
builder.add_edge("rule","place")
builder.add_edge("place","event")
builder.add_edge("event","item")
builder.add_edge("item","group")
builder.add_edge("group",END)

detail = builder.compile()


# FastAPI

In [37]:
app = FastAPI()
users : Dict[str, State] = {}
origins = ["https://ai0917-kt-aivle-shool-8th-bigprojec-eta.vercel.app",
          "hppt://localhost:8080"]
app.add_middleware(
CORSMiddleware,
allow_origins=origins,
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],  
allow_headers=origins,
)
@app.get("/")
def connect_check():
    return "AI 서버와 연결되었습니다."

#원문 저장 
@app.post("/novel_save")
async def process_novel_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    ep_num = data.get("ep_num")
    txt = data.get("txt")
    return save_text(user_id, work_id, ep_num, txt)

# 원문 읽기
@app.get("/novel_read")
async def process_novel_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    ep_num: int = Query(...)
):
    # 기존 코드의 txt 변수가 정의되어 있지 않아 에러가 날 수 있으니 확인이 필요해요!
    return open_text(user_id, work_id, ep_num)

#설정 저장
@app.post("/setting_save")
async def process_setting_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    lore_id = data.get("lore_id")
    setting = data.get("data")
    return save_json(user_id,work_id,lore_id,setting)

# 설정 읽기
@app.get("/setting_read")
async def process_setting_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    lore_id: str = Query(...)
):
    return open_json(user_id, work_id, lore_id)

#카테고리 추출 
@app.post("/categories")
async def process_cat(data : dict = Body(...)):
    global current_state
    initial_state : State = {
        "txt": open_text(data.get("user_id"),data.get("work_id"),data.get("ep_num")),
        "ep_num" : data.get("ep_num"),
        "subtitle" : data.get("subtitle"),
        "check" : dict(),
        "work_id" : data.get("work_id"),
        "user_id" : data.get("user_id"),
        "인물" : dict(),
        "세계" : dict(),
        "장소" : dict(),
        "사건" : dict(),
        "물건" : dict(),
        "집단" : dict(),
    }
    if initial_state['txt'] == "failed":
        return "Error 원문이 저장되어 있지 않습니다."
    work_id = initial_state.get('work_id')
    result = cat.invoke(initial_state)
    users[work_id] = result
    return result.get("check",{})

@app.post("/setting") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_setting(data : dict = Body(...)):
    work_id = data.get("work_id")
    categories = data.get("check", [])
    if work_id not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    user_state = users[work_id]  #기존의 데이/터 가져오기
    user_state["check"] = categories
    user_state = detail.invoke(user_state)
    result = comparison(user_state)
    return result 

@app.post("/comparison") 
async def process_get_comparison(data : dict = Body(...)):
    result = comparison(data)
    print(data)
    return result 

@app.post("/get_vector")
async def process_get_vector(data : dict = Body(...)):
    original = data.get("original")
    txt_original = f"{original}"
    return get_vector(txt_original)

#유사도 기반 vectorDB 검색
@app.post("/userQ")
async def process_userQ(data : dict = Body(...)):
    category = data.get("category")
    user_query = data.get("user_query")
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    sim = data.get("sim")
    limit = data.get("limit")
    return userQuestion(category,user_query,user_id,work_id,sim,limit)


@app.post("/dbupsert")
async def process_dbupsert(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    universe_id = data.get('universe_id')
    setting = data.get('setting')
    try:
        insert_setting = setting.get('신규 업로드')
        lorebooks_insert(universe_id,work_id,user_id,insert_setting)
        edit_settings = setting.get('설정 결합')
        lorebooks_update(edit_settings)
        users.pop(work_id,None)
    except Exception as e:
        return e
    return "성공"

@app.post("/relationship")
async def process_relationship(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    target = data.get('target',"*")
    result = relationship_select(target,work_id,user_id)
    result = Relationship(result)
    return result

@app.post("/timeline")
async def process_timeline(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    target = data.get('target',"*")
    result = timeline_select(target,work_id,user_id)
    result = timeline(result)
    return result

# 테스트함수

In [34]:
# a = """죄수들을 가둬 놓은 동굴 너머에는 거대한 구멍이 있었다. 가로만 30미터에 달하는 이 거친 수직 통로는 깊이가 수백 미터는 될 것 같았다.

# 케인은 가장자리에 서서 내려다보았다. 엄청난 범위의 암반이... '무언가'에 의해 뜯겨나간 채였다. 아르마다 슬링 전함의 주포로도 행성의 표면을 이렇게 깔끔하게 날려버릴 수는 없었다.

# 사라진 부분은 어디에 있는 것일까? 아예 없애버린 건가?

# "저 아래야." 나쿠리가 말했다.

# 케인은 수직 통로 안쪽 벽의 울퉁불퉁한 표면을 타고 내려가기 시작했다. 가까이서 보니 열기로 인해 생성된 것처럼 보였다. 튀어나온 바위는 윤이 나는 분홍색이었고, 잘 닦인 보석처럼 반짝거렸다. 하지만 위쪽 표면에는 두꺼운 모래층이 덮여 있었다. 이 구멍은 오래전에, 어쩌면 수천 년 전에 만들어진 것일지도 몰랐다. 불현듯, 케인의 머릿속에 뜨거운 금속 덩어리가 빙하에 떨어져 순식간에 얼음을 녹이며 깊은 구멍을 만들고, 금속이 지나간 자리가 다시 얼어붙어 반짝이는 장면이 떠올랐다.

# 하지만 암반에 구멍을 낼 수 있을까?

# 케인은 내려가면서 인터페이스로 탐사 스캔을 작동했다. 뒤따르던 나쿠리는 케인이 놀라서 한숨을 내뱉는 소리를 들었다.

# "역시." 그가 말했다.

# "이 결과가 정확한 건가?" 케인이 중얼거렸다.

# "그런 것 같아."

# "이건… 말이 안 돼." 케인은 말하며 인터페이스의 스캔을 다시 작동했다.

# "아니, 그럴 리 없어."

# "이건 마치..." 케인은 쉽게 설명할 방법을 찾지 못했다. 양자의 흔적은 기묘했다. 마치 다른 현실, 다른 공간 차원의 일부가 이 아이오난의 산과 순간적으로 교차하여 이곳의 존재를 완전히 지워버리고, 텅 빈 상처를 남겨둔 것 같았다.

# 미지의 무언가에 의해 찢겨나간 상처 말이다.

# "내가 왜 오디널을 찾았는지 알겠지?" 나쿠리가 물었다.

# 케인은 대답하지 않았다. 추측하는 중이었다. '공간 간 충돌의 결과였을까? 양자 이상 현상? 계획 아니면 우연? 이런 현상은 이론적으로만 가능하다. 아니면 슬링 드라이브 실패로 일어나는 희소하고 비극적인 결과일 것이다. 이것은 다중 우주 명제를 입증하는 증거가 될 수도 있다... '

# 나쿠리가 옳았다. 이것은 오디널이 '해야 할' 일이었고, 이미 높아진 케인의 직위도 크게 오를 것이었다. 획기적인 발견이었다. 데막시아 제국에서 가장 유명한 사람이 될 수 있는, 정말로 초고속 승진 가도를 달릴 수 있는 일이었다.

# 케인은 잠시 멈칫했다. 그것은 충격적인 생각이었다. 여기에서 처리해야 할 일이자 오디널의 의무였다. 정보를 평가하고, 분석하고, 수집하는 모든 업무는 제국의 이익을 위한 것이다. 그리고 이 공적을 세운 자의 이름을...

# 새로운 생각이 케인의 마음속에 스며들었다. 야망 어린 생각이 그를 방해했다. 조사 진행 계획을 세우려면 나쿠리와 의논해야 한다는 사실은 알고 있었다.

# 하지만 그러고 싶지 않았다. 혼자 진행하고 싶었다. 아무도, 심지어 나쿠리조차도 들이고 싶지 않았다. 어느 누구에게도 그럴 만한 자격이—

# 케인은 생각을 정리했다. 신디케이트, 기사단... 모두가 이곳으로 모인 것은 당연한 일이었다. 이건 엄청난 선물이었다. 다만...

# "...그들은 어떻게 알았지?" 케인이 질문했다.

# "뭐라고?"

# "난 네가 불러서 이곳에 왔어. 넌 기사단을 쫓다가 왔고. 그러면 기사단은 어떻게 오게 된 걸까?"

# "기사단도 알고 있었다고…?" 나쿠리가 조심스럽게 되물었다.

# "누가 알려준 건데?"

# "비밀 거래, 금기시된 설화... 죄다 말이 안 돼. 아니면 전설이나 신화라든가... 모르겠어, 보물 지도라도 있는 건가?"

# 이 말은 케인의 귀에도 공허하게 들렸다. 혹시, 만약, 누군가가 과거에 이곳을 발견했다면 그냥 지나치지 않았을 것이다. 그렇다면 이곳은 신성한 장소나 성지가 되었거나, 새로운 문화를 탄생시켰거나, 누군가를 황제로 추대했거나... 새로운 제국의 초석이 되었을지도 모른다.

# 아니다. 아무도 몰랐을 것이다. 기사단이 이곳에 온 것은 그저... 본능적인 일이었을 것이다.

# "그럼 신디케이트는?" 케인이 나쿠리에게 물었다.

# "신디케이트가 왜?"

# 케인은 생각했다. 자고는 이미 알고 있었다. 그 추잡한 기회주의자는 기사단의 존재조차 눈치채지 못했다. 자고가 여기 온 이유는 이것이었다. 게다가 자고는 모든 위험을 감수할 정도로 집요한 모습을 보였다. 대제국군과 마찰을 일으킬 정도로 말이다.

# 그는 '무언가'의 부름을 받고 드넓은 우주를 가로질러 이 변두리까지 왔다.

# 케인의 손바닥이 축축해졌다. 마지막 몇 미터를 더 미끄러져 내려가는 동안, 불안감은 점점 커졌다. 구덩이 바닥에 뭔가가 있었다. 마치 바닥에 녹아든 것처럼 보였다.

# "이건 도대체..."

# "우린 저게 원인일 거라고 생각했어." 나쿠리가 말했다. "이곳에 떨어져 구멍을 만든 거지…"

# 그의 목소리가 조금씩 작아졌다.

# "혹시 건드렸나?" 케인이 물었다.

# "아니, 아무도. 그럴 엄두도 못 냈지."

# 케인은 쭈그려 앉았다. 그 물체는 얕은 암석 단층에 묻힌 시커먼 화석처럼 보였다. 상상할 수도 없는 머나먼 고대에 묻혔던 뼈가 막 빛을 받아 드러난 것 같았다. 케인은 살짝 휘어진, 아름답게 장식된 긴 손잡이를 분간할 수 있었다. 머리에 달린 것은 거대한 날이었다. 인터페이스가 식별할 수 없는 미지의 금속으로 벼려진 손잡이와 날은 분명히 인간형 종족에 맞춰진 비율이었다.

# 낫이었다. 무기 말이다. 지금껏 알려진 어떤 문명에서도 발견된 적 없는 굉장한 유물이었다.

# 케인은 극도의 아름다움과 기괴함이 어떻게 동시에 공존할 수 있는지 궁금해졌다.

# 그리고 낮은 웃음소리가 들렸다. "무슨 일이야?" 케인이 나쿠리를 바라보며 물었다.

# "아무 말도 안 했는데." 나쿠리가 대답했다.

# 케인이 인터페이스를 건드려 보았지만 먹통이었다.

# "너무 깊이 들어왔나 보군." 나쿠리가 말했다. "이곳의 뭔가가 통신을 방해하고 있어."

# "올라가." 케인이 말했다. "'프랙털 쉬어'에 신호를 보내서 과학팀에게 탐지 장비를 모두 챙겨 모이라고 해. 두 시간 내로 이곳에 데려와. 여길 샅샅이 분석해서 마지막 정보 한 조각까지 남김없이 뽑아낸다."

# 나쿠리는 고개를 끄덕였지만 움직이지 않았다. "너 변했구나."

# "무슨 의미지?"

# "이제 오디널 다 됐군. 네 말투—"

# 케인이 코웃음 치며 말했다. "이럴 시간이 없어."

# "신디케이트 앞에서 한 행동은 뭐였는데? 덕분에 난 부하 네 명을 잃었어. 죽지 않을 수 있었던 네 명이 네 허세 때문에 죽었다고."

# "복잡한 상황이었잖아."

# "본대에 요청해 싹 쓸어 버릴 수도 있었어. 하지만 넌 네 오만함에 취해 있었지. 오디널 각하."

# "결국 필요한 정보를 얻었어." 케인이 말했다.

# "그리고 네 명이 죽었지."

# "사령관, 가서 함선에 신호를 보내. 두 번 말하지 않겠다."

# 나쿠리는 머뭇거렸다. "내가 널 부른 이유는... 그래, 내가 널 부른 건 내 것이 아니라는 걸 알았기 때문이야. 내 권한 밖이니까. 그러자 네 생각이 나더군. 너라면 뭘 해야 할지 알고 있을 것 같았지. 네겐 자격이 있을 거라고 생각했어."

# "'자격'이라고?"

# "이걸 차지할 자격 말이야! 나? 난 아니야, 난 그럴 만한 자격이…" 나쿠리는 케인을 바라보았다. "하지만 너라면 가능할 거라 생각했지. 그게 제국을 위한, 내 친구를 위한 도리인 줄 알았어. 하지만 이제 확실해졌군. 네가 어떤 존재인지. 네가 어떻게 변했는지 알아버렸으니까."

# 죽여 버려.

# 케인은 주위를 두리번거렸다. 누군가가 그에게 말을 걸었다.

# "여긴 우리뿐인가?" 케인이 속삭였다.

# "뭐라고?" 분노한 나쿠리가 되물었다.

# "사령관, 이곳에 보초를 배치했나?"

# "아니."

# "그렇다면 방금 말한 건 누구지?"

# "아무도 말하지 않았어!" 나쿠리는 딱딱하게 내뱉었다. "도대체 왜 그래? 이젠 나도 네가 누구인지 모르겠어!"

# "가서 함선에 신호를 보내. 지금 당장. 끝나는 대로 돌아와서 보고하도록."

# 나쿠리는 케인을 한번 노려보곤, 돌아서서 구덩이를 기어올랐다. 케인은 바닥에 박힌 무기 앞에 쭈그려 앉아 있었다.

# "방금 말한 거, 너지?" 케인이 질문했다.

# 너도 알지 않나. 내가 부르면 누군가가 듣고 오지. 나는 자격을 갖춘 자에게만 관심이 있다.

# "다들 자격 타령이군. 그래서 누가 자격이 있는데? 무슨 자격?"

# 날 소유할 자격이다. 누군가가 그 자신을 증명하면, 난 그자가 자격을 갖추었는지 알게 되지. 어쩌면 그게 그대일지도.

# "난 네가 누군지도 몰라."

# 나에 대해선 알 필요 없다. 내가 그대에 대해 알아야 하지. 나는 단 한 명의 적격자를 찾을 때까지 부를 것이다. 내가 부름을 멈출 때는 더 이상 부를 필요가 없을 때다.

# "난 제국의 오디널..."

# 난 그대가 무엇이든 상관 하지 않는다. 내가 관심을 가지는 것은 그대가 누구냐는 것이다. 그대의 야망. 그대의 꿈. 그대의 능력. 우주에 대한, 우주의 본분에 대한 그대의 생각.

# "난 오디널이다. 중요한 건 그것뿐이거든." 케인이 날카롭게 말했다. "내겐 해야 될 일이 있어. 임무 말이지."

# 못마땅한 임무, 갈수록 더 불만스러워질 임무 말이군. 그 남자를 향한 충성심은 약해지고 있다. 네가 생각하는 대의명분에 헌신하는 것은 소심한 행동이지. 아무도 네 생각을 알아주지 않고 아무도 네가 원하는 대로 행동하려 하지 않으니 날이 갈수록 좌절감은 커지겠지. 아무도 그럴 만한 힘이 없으니까.

# "내 의무는 데막시아 제국을 위해 이 장소를 확보하는 거다. 지금 골동품 무기와 대화하고 있다는 것도 믿기지 않는군. 양자 변이에 노출된 게 분명해. 이건 내 마음속이고, 내 정신에 뭔가 문제가 생긴 거야."

# 그러니까 지금 내가 환각이라는 말인가?

# "이곳은 이례적으로 뛰어난 과학적 가치를 가지고 있어. 넌 이곳의 중요한 유물이고. 난... 이곳에 있는 외계의 흔적이 만들어 낸 에너지 때문에 환청을 듣고 있는 게 분명해. 게다가—"

# 나쿠리가 사라진 지 꽤 오래 지나지 않았나?

# 케인은 벌떡 일어나 인터페이스의 크로노미터를 확인했다. 나쿠리가 나간 후로 대략 한 시간이 흘렀다. 한 시간이라고? 어떻게 시간이 그렇게 빨리 흐를 수 있지...?

# 시간은 언제든지 사라질 수 있는 환상이지.

# "내게 '자격'이 있다면 말이지?" 케인은 말을 내뱉고는 돌아서서 벽을 오르기 시작했다.

# 뒤에서 들려오는 낄낄거리는 웃음소리를 무시한 채.
# """
# a=json.dumps(a,ensure_ascii=False)[1:-1]
# print(a)

# 신규

In [35]:
# aa=timeline_select([1,2,3,4],15,"ssssssss")

In [36]:
# Relationship(relationship_select("키일로",15,"ssssssss"))